In [35]:
import numpy as np
import pandas as pd

In [36]:
# Example: select dataset
dataset_option = 'NASDAQ'  # or 'NASDAQ', 'Crypto'

# Build file paths
expected_returns_path = f'results/{dataset_option}/expected_returns.csv'
cov_matrix_path = f'results/{dataset_option}/cov_matrix.csv'

# Load data
expected_returns_df = pd.read_csv(expected_returns_path, index_col=0)
cov_matrix_df = pd.read_csv(cov_matrix_path, index_col=0)

In [37]:
assets = list(expected_returns_df.index)
expected_returns = expected_returns_df.values.flatten()
cov_matrix = cov_matrix_df.values

In [38]:
# Calculate Sharpe ratio for each asset (assuming risk-free rate = 0)
asset_volatility = np.sqrt(np.diag(cov_matrix))
sharpe_ratio = expected_returns / asset_volatility

In [39]:
# Get indices of top N assets by Sharpe ratio
N = 15  # or any number you want
sorted_indices = np.argsort(sharpe_ratio)[::-1][:N]
top_assets = np.array(assets)[sorted_indices]

In [40]:
# Filter expected returns and covariance matrix for top assets
mu = expected_returns[sorted_indices]
Sigma = cov_matrix[np.ix_(sorted_indices, sorted_indices)]
n_assets = N
portfolio_size = 5  # Number of assets to select

In [41]:
import cvxpy as cp

In [42]:
# Variables
w = cp.Variable(n_assets)
z = cp.Variable(n_assets, boolean=True)

In [43]:
# Constraints
constraints = [
    cp.sum(w) == 1,
    w >= 0,
    w <= z,
    cp.sum(z) == portfolio_size
]

In [44]:
min_weight = 1.0 / portfolio_size  # or a smaller value, e.g., 0.05
constraints += [w >= min_weight * z]

In [45]:
# UI selection: 'low', 'medium', or 'high'
gamma_option = 'medium'  # Change as needed

# Map UI options to gamma values
gamma_map = {
    'low': 0.1,
    'medium': 0.5,
    'high': 1.0
}

gamma = gamma_map.get(gamma_option, 0.5)  # Default to 0.5 if not found
print(f"Gamma value set to: {gamma}")

Gamma value set to: 0.5


In [46]:
# Objective: maximize expected return with risk penalty

objective = cp.Maximize(mu @ w - gamma * cp.quad_form(w, Sigma))

In [47]:
# %pip install ecos

In [48]:
# Problem
prob = cp.Problem(objective, constraints)
prob.solve(solver=cp.ECOS_BB)

np.float64(0.0053526594832444055)

In [49]:
# Results
selected_assets = [top_assets[i] for i in range(n_assets) if z.value[i] > 0.5]
weights = w.value

In [50]:
print("Selected assets:", selected_assets)
print("Weights:", weights)

Selected assets: [np.str_('AppLovin Corporation'), np.str_('Palantir Technologies Inc.'), np.str_('MicroStrategy Incorporated'), np.str_('Axon Enterprise, Inc.'), np.str_('NVIDIA Corporation')]
Weights: [2.00000000e-01 2.00000000e-01 7.50260337e-10 8.07034490e-10
 2.00000000e-01 1.58885084e-09 2.00000000e-01 1.99999974e-01
 1.81071372e-08 8.46124194e-10 1.21087221e-09 1.03113692e-09
 7.30127362e-10 1.21884660e-09 6.51592744e-10]


In [51]:
# ---- Portfolio Weights based on Expected Returns ----

# Extract chosen assets (from classical optimization solution)
selected_indices = [i for i in range(n_assets) if z.value[i] > 0.5]

chosen_returns = mu[selected_indices]
chosen_assets = top_assets[selected_indices]

# Compute proportional weights
chosen_weights = chosen_returns / chosen_returns.sum()

# Total investment
total_investment = 1_000_000
allocation = chosen_weights * total_investment

# Create allocation DataFrame
portfolio_allocation = pd.DataFrame({
    "Asset": chosen_assets,
    "Expected Return": chosen_returns,
    "Weight": chosen_weights,
    "Investment (₹)": allocation
})

print("\nPortfolio Allocation (Proportional to Expected Returns):")
print(portfolio_allocation)


Portfolio Allocation (Proportional to Expected Returns):
                        Asset  Expected Return    Weight  Investment (₹)
0        AppLovin Corporation         0.007192  0.251990   251989.531896
1  Palantir Technologies Inc.         0.005579  0.195485   195484.945984
2  MicroStrategy Incorporated         0.008502  0.297891   297891.350214
3       Axon Enterprise, Inc.         0.003480  0.121917   121916.923680
4          NVIDIA Corporation         0.003788  0.132717   132717.248225
